In [0]:
%sql
CREATE OR REPLACE TABLE silver.customers AS 
SELECT 
    customer_id,
    name,
    email,
    CAST(usd_balance AS DOUBLE) AS usd_balance_original,  -- Mantendo saldo original de USD
    CAST(btc_balance AS DOUBLE) AS btc_balance_original,  -- Mantendo saldo original de BTC
    CAST(last_update AS TIMESTAMP) AS last_update
FROM bronze.customers
WHERE name NOT IN ('Mark Cunningham', 'Mark Savage');  -- Removendo clientes inválidos

In [0]:
%sql
CREATE OR REPLACE TABLE silver.transactions AS 
SELECT 
    transaction_id,
    customer_id,
    CAST(transaction_type AS STRING) AS transaction_type,  -- Forçando para STRING
    btc_amount,
    transaction_date
FROM bronze.transactions
WHERE DATE(transaction_date) BETWEEN '2025-01-29' AND '2025-01-30'; -- Regra de transações válidas

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE silver.bitcoin_price AS 
SELECT 
    datetime AS price_timestamp,
    currency,
    amount AS btc_price
FROM bronze.bitcoin_price;

In [0]:
%sql
CREATE OR REPLACE TABLE gold.transactions AS 
WITH btc_price AS (
    SELECT 
        price_timestamp, 
        btc_price,
        LAG(price_timestamp) OVER (ORDER BY price_timestamp) AS prev_timestamp
    FROM silver.bitcoin_price
)
SELECT 
    t.transaction_id,
    t.customer_id,
    t.btc_amount,
    t.transaction_type,
    -- Valor da transação em USD (compra negativa, venda positiva)
    ROUND(
        CASE 
            WHEN t.transaction_type = 'compra' THEN (t.btc_amount * p.btc_price) * -1  
            ELSE (t.btc_amount * p.btc_price)  
        END, 2
    ) AS transaction_value_in_usd,
    t.transaction_date
FROM silver.transactions t
JOIN btc_price p
    ON t.transaction_date BETWEEN p.prev_timestamp AND p.price_timestamp;

In [0]:
%sql
CREATE OR REPLACE TABLE gold.customers AS 
SELECT 
    c.customer_id,
    c.name,
    c.email,
    c.usd_balance_original,  -- Mantendo saldo original de USD
    c.btc_balance_original,  -- Mantendo saldo original de BTC
    -- Calculando saldo atualizado de BTC considerando compras e vendas
    c.btc_balance_original + COALESCE(SUM(
        CASE 
            WHEN t.transaction_type = 'compra' THEN t.btc_amount  -- Se comprou, adiciona BTC
            WHEN t.transaction_type = 'venda' THEN -t.btc_amount  -- Se vendeu, reduz BTC
            ELSE 0
        END
    ), 0) AS btc_balance_final,
    COUNT(t.transaction_id) AS total_transactions,
    -- Calculando total de USD gasto somando transaction_value_in_usd com usd_balance_original
    c.usd_balance_original + COALESCE(SUM(t.transaction_value_in_usd), 0) AS usd_balance_final
FROM silver.customers c
LEFT JOIN gold.transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id, c.name, c.email, c.usd_balance_original, c.btc_balance_original;